In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import config as creds
from path import Path
from sklearn.model_selection import train_test_split

# import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 

In [2]:
# Create connection string to PostgreSQL database using credentials stored in a config file
conn_string = "postgresql://"+ creds.PGUSER +":"+ creds.PGPASSWORD +"@"+ creds.PGHOST +"/"+ creds.PGDATABASE

In [3]:
# Create an engine instance with sqlalchemy
db = create_engine(conn_string)
conn = db.connect()

In [4]:
fake_news_df = pd.read_sql('SELECT * FROM mlenv_ready', conn)

In [5]:
# Close the connection
conn.close()

In [6]:
fake_news_df

,uuid,site_url,author,published,type,has_image,spam_score,replies_count,participants_count,likes,comments,shares,label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,100percentfedup.com,Barracuda Brigade,2016-10-26,bias,1.0,0.000,0,1,0,0,0,Real
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,100percentfedup.com,reasoning with facts,2016-10-29,bias,1.0,0.000,0,1,0,0,0,Real
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,100percentfedup.com,Barracuda Brigade,2016-10-31,bias,1.0,0.000,0,1,0,0,0,Real
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,100percentfedup.com,Fed Up,2016-11-01,bias,1.0,0.068,0,0,0,0,0,Real
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,100percentfedup.com,Fed Up,2016-11-01,bias,1.0,0.865,0,0,0,0,0,Real
...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,dd356f03a743b795c3ebe8562160cf3347f677d6,wnd.com,-NO AUTHOR-,2016-10-27,bias,0.0,0.000,0,1,0,0,0,Real
972,37903d3104f269c41250752b30d469189f86222e,wnd.com,-NO AUTHOR-,2016-10-27,bias,0.0,0.000,0,1,0,0,0,Real
973,a631576d9ebcdc01c623a7cbdd1c370245b17433,wnd.com,Joseph Farah,2016-10-27,bias,0.0,0.000,0,1,0,0,0,Real
974,9eea614efdad6d5ef6a8c7f2ec4079aa60eecdfa,wnd.com,-NO AUTHOR-,2016-10-27,bias,0.0,0.000,0,1,0,0,0,Real


In [7]:
# Drop the non-beneficial uuid columns

fake_news_df.drop(columns=["uuid","published" ], inplace=True)
fake_news_df.head()

,site_url,author,type,has_image,spam_score,replies_count,participants_count,likes,comments,shares,label
0,100percentfedup.com,Barracuda Brigade,bias,1.0,0.000,0,1,0,0,0,Real
1,100percentfedup.com,reasoning with facts,bias,1.0,0.000,0,1,0,0,0,Real
2,100percentfedup.com,Barracuda Brigade,bias,1.0,0.000,0,1,0,0,0,Real
3,100percentfedup.com,Fed Up,bias,1.0,0.068,0,0,0,0,0,Real
4,100percentfedup.com,Fed Up,bias,1.0,0.865,0,0,0,0,0,Real


In [8]:
# Generate our categorical variable lists
fakenews_cat = fake_news_df.dtypes[fake_news_df.dtypes == "object"].index.tolist()
fakenews_cat

['site_url', 'author', 'type', 'label']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fake_news_df[fakenews_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(fakenews_cat)
encode_df.head()

/Users/charlotte.rotner/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,site_url_100percentfedup.com,site_url_21stcenturywire.com,site_url_abcnews.com.co,site_url_abeldanger.net,site_url_abovetopsecret.com,site_url_activistpost.com,site_url_adobochronicles.com,site_url_ahtribune.com,site_url_americannews.com,site_url_americasfreedomfighters.com,...,type_bias,type_bs,type_conspiracy,type_fake,type_hate,type_junksci,type_satire,type_state,label_Fake,label_Real
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Merge one-hot encoded features and drop the originals
fake_news_df = fake_news_df.merge(encode_df,left_index=True, right_index=True)
fake_news_df = fake_news_df.drop(columns = fakenews_cat)


In [11]:
fake_news_df = fake_news_df.drop(['label_Real'], axis=1)
fake_news_df.head()

,has_image,spam_score,replies_count,participants_count,likes,comments,shares,site_url_100percentfedup.com,site_url_21stcenturywire.com,site_url_abcnews.com.co,...,author_xuenchen,type_bias,type_bs,type_conspiracy,type_fake,type_hate,type_junksci,type_satire,type_state,label_Fake
0,1.0,0.000,0,1,0,0,0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000,0,1,0,0,0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.000,0,1,0,0,0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.068,0,0,0,0,0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.865,0,0,0,0,0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
fake_news_df.rename(columns={"label_Fake":"Fake"}, inplace=True)

In [13]:
fake_news_df['Fake'].value_counts()

1.0    626
0.0    350
Name: Fake, dtype: int64

 ## Separate the Features (X) from the Target (y)

In [14]:
y = fake_news_df["Fake"]
X = fake_news_df.drop(columns="Fake")

 ## Split our data into training and testing

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(732, 357)

 ## Create a Logistic Regression Model

In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

 ## Fit (train) or model using the training data

In [17]:
classifier.fit(X_train, y_train)

/Users/charlotte.rotner/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

 ## Make predictions

In [18]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0.0,0.0
1,1.0,1.0
2,1.0,1.0
3,0.0,0.0
4,1.0,1.0
5,0.0,0.0
6,1.0,1.0
7,0.0,0.0
8,0.0,0.0
9,1.0,1.0


In [19]:
# Generate accuracy score 
from sklearn.metrics import confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))

1.0


In [23]:
# Generate a classification report 
report = classification_report(y_test, y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        87
         1.0       1.00      1.00      1.00       157

    accuracy                           1.00       244
   macro avg       1.00      1.00      1.00       244
weighted avg       1.00      1.00      1.00       244

